# Compare coordinate descent SDP with FANOK

+ FANOK package: https://github.com/qrebjock/fanok
+ FANOK API (need to call `fanok.sdp.sdp.sdp_full`): https://fanok.readthedocs.io/en/latest/fanok.sdp.html#module-fanok.sdp

In [1]:
using Revise
using Knockoffs
using Random
using GLMNet
using Distributions
using LinearAlgebra
using ToeplitzMatrices
using StatsBase
using PyCall
using BenchmarkTools
# using Plots
# gr(fmt=:png);

# import FANOK functions
py"""
from sklearn.datasets import make_regression
from fanok import GaussianKnockoffs, KnockoffSelector
from fanok.statistics import EstimatorStatistics
from fanok.sdp.sdp import sdp_full
"""

This is example 1 from https://amspector100.github.io/knockpy/mrcknock.html
where we generate Gaussian X with covariance structure

\begin{aligned}
\Sigma = 
\begin{pmatrix}
    1 & \rho & \rho & ... & \rho\\
    \rho & 1 & & ... & \rho\\
    \vdots & & & 1 & \vdots \\
    \rho & \cdots & & & 1
\end{pmatrix}
\end{aligned}
SDP knockoffs are provably powerless in this situation.

First, simulate data

In [3]:
seed = 2022

# simulate cov
n = 1000
p = 200
ρ = 0.4
Σ = Matrix(SymmetricToeplitz(ρ.^(0:(p-1)))) # true covariance matrix

200×200 Matrix{Float64}:
 1.0          0.4          0.16         …  1.61391e-79  6.45562e-80
 0.4          1.0          0.4             4.03477e-79  1.61391e-79
 0.16         0.4          1.0             1.00869e-78  4.03477e-79
 0.064        0.16         0.4             2.52173e-78  1.00869e-78
 0.0256       0.064        0.16            6.30432e-78  2.52173e-78
 0.01024      0.0256       0.064        …  1.57608e-77  6.30432e-78
 0.004096     0.01024      0.0256          3.9402e-77   1.57608e-77
 0.0016384    0.004096     0.01024         9.8505e-77   3.9402e-77
 0.00065536   0.0016384    0.004096        2.46263e-76  9.8505e-77
 0.000262144  0.00065536   0.0016384       6.15656e-76  2.46263e-76
 0.000104858  0.000262144  0.00065536   …  1.53914e-75  6.15656e-76
 4.1943e-5    0.000104858  0.000262144     3.84785e-75  1.53914e-75
 1.67772e-5   4.1943e-5    0.000104858     9.61963e-75  3.84785e-75
 ⋮                                      ⋱               
 1.53914e-75  3.84785e-75  9.61963e-

Run FANOK's coordiate descent SDP

In [4]:
# put X and y into python by appending $, so like $y and $X
@time begin
    py"""
    s, objective = sdp_full($Σ, return_objectives=True)
    """
end

  0.459536 seconds (614.31 k allocations: 35.663 MiB, 4.59% gc time, 57.87% compilation time)


Julia (standard) SDP

In [11]:
@time strue = Knockoffs.solve_SDP(Σ);

  4.623247 seconds (272.24 k allocations: 186.864 MiB, 1.61% gc time)


Julia SDP via coordiate descent

In [12]:
@time s_julia = Knockoffs.solve_sdp_fast(Σ);

  0.161295 seconds (9 allocations: 630.500 KiB)


Compare solutions

In [13]:
[strue s2 py"s"]

200×3 Matrix{Float64}:
 1.0       1.0       1.0
 0.971429  0.960872  0.960873
 0.811429  0.828497  0.828496
 0.875429  0.865028  0.86503
 0.849829  0.855096  0.855093
 0.860069  0.85764   0.857643
 0.855973  0.857036  0.857034
 0.857611  0.857161  0.857162
 0.856956  0.857143  0.857142
 0.857218  0.857142  0.857142
 0.857113  0.857144  0.857144
 0.857155  0.857143  0.857143
 0.857138  0.857143  0.857143
 ⋮                   
 0.857155  0.85711   0.857142
 0.857113  0.857112  0.857134
 0.857218  0.85718   0.857134
 0.856956  0.857246  0.857162
 0.857611  0.857101  0.857189
 0.855973  0.856794  0.857098
 0.860069  0.857416  0.856538
 0.849829  0.858843  0.860041
 0.875429  0.851521  0.850427
 0.811429  0.857665  0.858348
 0.971429  0.939725  0.939462
 1.0       1.0       1.0

In [14]:
py"objective"

60-element Vector{Real}:
   0
  18.958965074495318
  63.501034948920974
  93.80171448516475
 116.4563790661536
 132.9152003436284
 144.6896562798062
 153.0750934460021
 158.99136155116514
 163.1154937543466
 165.9625188021524
 167.90874189848859
 169.2279721794348
   ⋮
 171.8746870606135
 171.87469455827267
 171.87470053845374
 171.87470531111322
 171.8747091218898
 171.8747121658061
 171.87471459792832
 171.87471654169894
 171.87471809548202
 171.87471933771928
 171.87472033100372
 171.874721125308

## Conclusion

+ Julia CCD routine is ~3x faster than FANOK's CCD routine for solving SDP problems
+ Julia CCD is roughly ~30x faster than calling a SDP solver (Hypatia + JuMP in this case)